## Librerias

import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata


In [26]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import datetime as dt
import warnings
import re, unicodedata

In [27]:
# Fijar directorio
DATA_DIR = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\raw")
DATA_OUT = Path(rf"C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed")

In [28]:
# Lista todos los archivos .csv
csv_files = sorted(DATA_DIR.glob("*.csv"))
print(f"Se listan {len(csv_files)} archivos a procesar:")
for f in csv_files:
    print("-", f.name)

## Lectura de datos
data_raw = []
cols_by_file = {}
for file in csv_files:
    df = pd.read_csv(file, sep="|")
    df["file_origin"] = file.name
    data_raw.append(df)
    cols_by_file[file.name] = set(df.columns)
    
print("\nproceso cargado!")

print("\nUniendo datos!")
raw_all = pd.concat(data_raw, ignore_index=True)
print("Filas totales:", len(raw_all))
print("Columnas:", list(raw_all.columns))

Se listan 19 archivos a procesar:
- data_alquiler_departamentos_adondevivir_Lima Centro.csv
- data_alquiler_departamentos_adondevivir_Lima Este.csv
- data_alquiler_departamentos_adondevivir_Lima Moderna.csv
- data_alquiler_departamentos_adondevivir_Lima Norte.csv
- data_alquiler_departamentos_adondevivir_Lima Sur.csv
- data_alquiler_departamentos_adondevivir_Lima Top.csv
- data_venta_casas_adondevivir_Lima Centro.csv
- data_venta_casas_adondevivir_Lima Este.csv
- data_venta_casas_adondevivir_Lima Moderna.csv
- data_venta_casas_adondevivir_Lima Norte.csv
- data_venta_casas_adondevivir_Lima Sur.csv
- data_venta_casas_adondevivir_Lima Top.csv
- data_venta_departamentos_adondevivir_Lima Centro.csv
- data_venta_departamentos_adondevivir_Lima Este.csv
- data_venta_departamentos_adondevivir_Lima Moderna.csv
- data_venta_departamentos_adondevivir_Lima Norte.csv
- data_venta_departamentos_adondevivir_Lima Sur.csv
- data_venta_departamentos_adondevivir_Lima Top.csv
- data_venta_departamentos_urb

In [29]:
print("\nLimpieza de datos!")

## Limpieza de datos - duplicados Otros. 

dups = raw_all["enlace"].duplicated(keep=False)
print("Registros duplicados por enlace:", dups.sum())

# ¿Cuántos enlaces únicos?
print("Enlaces únicos:", raw_all["enlace"].nunique())

## removeremos duplicados.
raw_all_nd = raw_all.drop_duplicates(subset=["enlace"], keep="first") 
print("se tiene solo enlaces unicos:", raw_all_nd["enlace"].nunique())


Limpieza de datos!
Registros duplicados por enlace: 347
Enlaces únicos: 36907
se tiene solo enlaces unicos: 36907


In [30]:
raw_all_nd.columns

Index(['fuente', 'inmueble', 'operacion', 'zona', 'distrito', 'direccion',
       'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin'],
      dtype='object')

In [31]:
print("\nDistrito y Direccion correctos?")


Distrito y Direccion correctos?


In [32]:
valores = {
    "Brena": "Breña",
    "Jesus Maria": "Jesús María",
    "San Martin De Porres": "San Martín de Porres",
    "Lurin": "Lurín",
    "Ancon": "Ancón",
    "Santa Maria Del Mar": "Santa María del Mar",
    "Villa Maria Del Triunfo": "Villa María del Triunfo",
}

s = (raw_all_nd["distrito"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True))

raw_all_nd["distrito_2"] = s.replace(valores)   # crea la nueva columna

raw_all_nd["distrito_norm"] = raw_all_nd["distrito_2"].str.lower().str.strip()
raw_all_nd["direccion_norm"] = raw_all_nd["direccion"].str.lower().str.strip()

raw_all_nd["distrito_en_direccion"] = raw_all_nd.apply(
    lambda row: row["distrito_norm"] in row["direccion_norm"],
    axis=1
)

C:\Users\PC\AppData\Local\Temp\ipykernel_28708\3954129704.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito_2"] = s.replace(valores)   # crea la nueva columna
C:\Users\PC\AppData\Local\Temp\ipykernel_28708\3954129704.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_all_nd["distrito_norm"] = raw_all_nd["distrito_2"].str.lower().str.strip()
C:\Users\PC\AppData\Local\Temp\ipykernel_28708\3954129704.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [33]:
cols = [ #"distrito_2", "direccion", 
        "distrito_norm", "direccion_norm", "distrito_en_direccion"]
raw_all_nd[raw_all_nd["distrito_en_direccion"]==False][cols]
df_arreglar = raw_all_nd[raw_all_nd["distrito_en_direccion"]==False].copy()##[["distrito", "direccion", "enlace"]]

canon = {
    # canónico : [alias que podrían aparecer en la dirección]
    "Ate Vitarte": ["ate vitarte", "ate"],
    "Breña": ["breña", "brena"],
    "Carabayllo": ["carabayllo"],
    "Chaclacayo": ["chaclacayo"],
    "El Agustino": ["el agustino", "agustino"],
    "Independencia": ["independencia"],
    "Jesús María": ["jesus maria", "jesús maria", "jesus maría"],
    "Chosica Lurigancho": [ "chosica"], ##"chosica lurigancho", "lurigancho-chosica",
    "Lurín": ["lurin", "lurín"],
    "Pachacámac": ["pachacamac", "pachacámac"],
    "Pucusana": ["pucusana"],
    "Puente Piedra": ["puente piedra"],
    "Punta Negra": ["punta negra"],
    "Rímac": ["rimac", "rímac"],
    "San Juan De Miraflores": ["san juan de miraflores", "sjm"],
    "San Luis": ["san luis"],
    "San Martín De Porres": ["san martin de porres", "san martín de porres", "smp"],
    "Villa María Del Triunfo": ["villa maria del triunfo", "villa maría del triunfo", "vmt"],
    "Ancón": ["ancon", "ancón"],
    "Lince": ["lince"],
    "Santa Rosa": ["santa rosa"],
    "Barranco": ["barranco"],
    "Magdalena": ["magdalena del mar", "magdalena"],
    "Punta Hermosa": ["punta hermosa"],
    "Santiago De Surco": ["santiago de surco", "surco"],
    "San Juan De Lurigancho": ["san juan de lurigancho"],
    "Lima Cercado" : ["cercado", "cercado de lima", "lima cercado"],
    "Los Olivos" : ["los olivos"],
    "Comas" : ["comas"],
    "San Bartolo" : ["san bartolo"], 
    'Villa El Salvador' : ["villa el salvador"],
    "Santa Maria del Mar": ["santa maria del mar"],
    "San Isidro" : ["san isidro"],
    "La Molina" : ["la molina"]
    # agrega aquí el resto hasta completar 43 distritos
}

In [34]:

def strip_accents(s):
    if not isinstance(s, str):
        return s
    return ''.join(ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn")

df_arreglar["dir_norm"]  = df_arreglar["direccion"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)
df_arreglar["dist_norm"] = df_arreglar["distrito_2"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).map(strip_accents)

# precompila patrones regex alias → canónico
patterns = []
for canon_name, aliases in canon.items():
    for alias in aliases:
        pat = re.compile(rf"\b{re.escape(strip_accents(alias))}\b")
        patterns.append((canon_name, pat))

def candidates_from_address(addr_norm: str):
    if not isinstance(addr_norm, str):
        return []
    hits = []
    for canon_name, pat in patterns:
        if pat.search(addr_norm):
            hits.append(canon_name)
    # quitar duplicados preservando orden
    return  list(dict.fromkeys(hits))

df_arreglar["dist_candidates"] = df_arreglar["dir_norm"].map(candidates_from_address)
df_arreglar["distrito_candidato"] = df_arreglar["dist_candidates"].str.get(0) 

In [35]:
raw_all_nd.columns

Index(['fuente', 'inmueble', 'operacion', 'zona', 'distrito', 'direccion',
       'precio', 'soles', 'dolares', 'variacion', 'mantenimiento',
       'caracteristica', 'area', 'dormitorio', 'baños', 'estacionamientos',
       'detalle', 'enlace', 'fecha', 'file_origin', 'distrito_2',
       'distrito_norm', 'direccion_norm', 'distrito_en_direccion'],
      dtype='object')

In [36]:

data_p1 = raw_all_nd[raw_all_nd["distrito_en_direccion"]==True]

cols_obj =['fuente', 'inmueble', 'operacion', 'distrito', 'zona', 'direccion', 'precio',
            'soles', 'dolares', 'variacion', 'mantenimiento', 'caracteristica',
            'area', 'dormitorio', 'baños', 'estacionamientos', 'detalle', 'enlace',
            'fecha', 'file_origin', 'distrito_candidato', 'distrito_norm',
            'direccion_norm', 'distrito_en_direccion']

data_p2 = df_arreglar[df_arreglar["distrito_candidato"].notna()][cols_obj].copy()

data_p2.columns = ['fuente', 'inmueble', 'operacion', 'distrito', 'zona', 'direccion', 'precio',
            'soles', 'dolares', 'variacion', 'mantenimiento', 'caracteristica',
            'area', 'dormitorio', 'baños', 'estacionamientos', 'detalle', 'enlace',
            'fecha', 'file_origin', 'distrito_2', 'distrito_norm',
            'direccion_norm', 'distrito_en_direccion']

data_vivienda = pd.concat([data_p1, data_p2], axis=0)

def normalizar(texto: str) -> str:
    # 1. Pasar a minúsculas
    texto = texto.lower()
    # 2. Quitar acentos
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )
    # 3. Quitar dobles espacios y limpiar
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto


data_vivienda["distrito_oficial"] = data_vivienda["distrito_2"].apply(normalizar)

In [94]:

print("\nCreacion de variables!")

cols_analisis = ['fuente','inmueble','operacion',
                'distrito','distrito_2', 'distrito_oficial', 'zona',
                'direccion','precio',
                'soles','dolares','variacion','mantenimiento','caracteristica',
                'area','dormitorio','baños','estacionamientos','detalle','enlace',
                'fecha','file_origin']
data_analisis = data_vivienda[cols_analisis].copy()

# Normalizaciones básicas
data_analisis["fuente"] = data_analisis["fuente"].astype(str).str.strip().str.lower()
data_analisis["distrito_final"] = data_analisis["distrito_oficial"].astype(str).str.strip().str.title()
data_analisis["inmueble"] = data_analisis["inmueble"].astype(str).str.strip().str.lower()
data_analisis["operacion"] = data_analisis["operacion"].astype(str).str.strip().str.lower()
data_analisis["precio_pen"] = data_analisis["soles"]
data_analisis["precio_usd"] = data_analisis["dolares"]

## solo registros con precios
data_analisis["sin_precio"] = (data_analisis["precio_pen"].isna() & data_analisis["precio_usd"].isna()).astype(int)
data_analisis = data_analisis.query("sin_precio == 0").copy()

# AREA robusta
tmp = data_analisis["area"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mm = tmp.str.split(" a ", n=1, expand=True)
data_analisis["area_min"] = pd.to_numeric(mm[0].str.replace(",", "."), errors="coerce")
data_analisis["area_max"] = pd.to_numeric(mm[1].str.replace(",", "."), errors="coerce")
data_analisis["area_promedio"] = data_analisis[["area_min","area_max"]].mean(axis=1)

# DORMITORIO robusto
tmpd = data_analisis["dormitorio"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
mmd = tmpd.str.split(" a ", n=1, expand=True)
data_analisis["domitorio_min"] = pd.to_numeric(mmd[0], errors="coerce")
data_analisis["domitorio_max"] = pd.to_numeric(mmd[1], errors="coerce")

# Precio por m2 (división segura)
den = data_analisis["area_promedio"].replace(0, np.nan)
data_analisis["precio_m2_pen"] = (data_analisis["precio_pen"] / den).round(2)
data_analisis["precio_m2_usd"] = (data_analisis["precio_usd"] / den).round(2)

# Flags
data_analisis["tiene_mantenimiento"] = data_analisis["mantenimiento"].notna().astype(int)
data_analisis["tiene_estacionamientos"] = data_analisis["estacionamientos"].notna().astype(int)

data_analisis["banios"] = data_analisis["caracteristica"].str.extract(r"(\d+)\s*bañ").astype("Int64")

data_analisis.head()


Creacion de variables!


,fuente,inmueble,operacion,distrito,distrito_2,distrito_oficial,zona,direccion,precio,soles,...,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,precio_m2_usd,tiene_mantenimiento,tiene_estacionamientos,banios
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,...,66.0,NaN,66.0,3.0,NaN,27.27,NaN,1,0,2
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,...,70.0,NaN,70.0,3.0,NaN,28.57,8.21,1,0,2
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,...,60.0,NaN,60.0,3.0,NaN,30.00,NaN,1,0,2
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,...,70.0,NaN,70.0,3.0,NaN,28.57,NaN,1,0,2
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,...,68.0,NaN,68.0,3.0,NaN,26.47,NaN,0,1,2


In [59]:
data_analisis.head()

,fuente,inmueble,operacion,distrito,distrito_2,distrito_oficial,zona,direccion,precio,soles,...,area_min,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,precio_m2_usd,tiene_mantenimiento,tiene_estacionamientos,banios
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,...,66.0,NaN,66.0,3.0,NaN,27.27,NaN,1,0,2
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,...,70.0,NaN,70.0,3.0,NaN,28.57,8.21,1,0,2
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,...,60.0,NaN,60.0,3.0,NaN,30.00,NaN,1,0,2
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,...,70.0,NaN,70.0,3.0,NaN,28.57,NaN,1,0,2
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,...,68.0,NaN,68.0,3.0,NaN,26.47,NaN,0,1,2


In [60]:
data_analisis["distrito_2"].unique()

array(['Lima Cercado', 'Breña', 'La Victoria', 'Rimac', 'San Luis',
       'Ate Vitarte', 'Chaclacayo', 'Santa Anita',
       'San Juan de Lurigancho', 'Jesús María', 'Lince', 'Magdalena',
       'San Miguel', 'Pueblo Libre', 'Surquillo', 'Carabayllo', 'Comas',
       'Independencia', 'Los Olivos', 'Puente Piedra',
       'San Martin de Porres', 'Ancón', 'Chorrillos', 'Lurín',
       'Pachacamac', 'San Juan de Miraflores', 'Villa El Salvador',
       'Pucusana', 'Punta Hermosa', 'Punta Negra', 'San Bartolo',
       'Santa Maria del Mar', 'Miraflores', 'San Isidro', 'La Molina',
       'Santiago de Surco', 'San Borja', 'Barranco', 'Cieneguilla',
       'El Agustino', 'Santa Rosa', 'Villa Maria del Triunfo', 'Rímac',
       'San Martín De Porres', 'Chosica Lurigancho',
       'San Juan De Lurigancho', 'Pachacámac', 'Santiago De Surco',
       'San Juan De Miraflores', 'Villa María Del Triunfo'], dtype=object)

In [72]:
zonas_lima = {
    'Lima Top': ['Miraflores', 'San Isidro', 'La Molina', 'Santiago de Surco', 'San Borja', 'Barranco', 'Santiago De Surco'],
    'Lima Moderna': ['Jesús María', 'Lince', 'Magdalena', 'San Miguel', 'Pueblo Libre', 'Surquillo'],
    'Lima Centro': ['Lima Cercado', 'Breña', 'La Victoria', 'Rimac', 'Rímac', 'San Luis'],
    'Lima Este': ['Ate Vitarte', 'Cieneguilla', 'Chaclacayo', 'Chosica Lurigancho', 'Santa Anita', 'El Agustino', 'San Juan de Lurigancho', 'San Juan De Lurigancho'],
    'Lima Norte': ['Carabayllo', 'Comas', 'Independencia', 'Los Olivos', 'Puente Piedra', 'San Martín de Porres', 'San Martín De Porres', 'San Martin de Porres', 'Ancón', 'Santa Rosa'],
    'Lima Sur': ['Chorrillos', 'Lurín', 'Pachacámac', 'Pachacamac', 'San Juan de Miraflores', 'San Juan De Miraflores', 'Villa El Salvador', 'Villa Maria del Triunfo', 'Villa María Del Triunfo'
                , 'Pucusana', 'Punta Hermosa', 'Punta Negra', 'San Bartolo', 'Santa Maria del Mar']
}

# Se crea un mapeo inverso (distrito -> zona) para una asignación eficiente.
distrito_a_zona = {distrito: zona for zona, distritos_en_zona in zonas_lima.items() for distrito in distritos_en_zona}
# Se aplica el mapeo para crear la nueva columna.
data_analisis['distrito_categoria'] = data_analisis['distrito_2'].map(distrito_a_zona).fillna('Otra Zona')
data_analisis['distrito_categoria'] = data_analisis['distrito_categoria'].astype('category')

In [70]:
data_analisis.head()

,fuente,inmueble,operacion,distrito,distrito_2,distrito_oficial,zona,direccion,precio,soles,...,area_max,area_promedio,domitorio_min,domitorio_max,precio_m2_pen,precio_m2_usd,tiene_mantenimiento,tiene_estacionamientos,banios,distrito_categoria
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,...,NaN,66.0,3.0,NaN,27.27,NaN,1,0,2,Lima Centro
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,...,NaN,70.0,3.0,NaN,28.57,8.21,1,0,2,Lima Centro
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,...,NaN,60.0,3.0,NaN,30.00,NaN,1,0,2,Lima Centro
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,...,NaN,70.0,3.0,NaN,28.57,NaN,1,0,2,Lima Centro
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Cercado,lima cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,...,NaN,68.0,3.0,NaN,26.47,NaN,0,1,2,Lima Centro


In [73]:
data_analisis[data_analisis['distrito_categoria'] == 'Otra Zona']['distrito_2'].unique()

array([], dtype=object)

In [74]:
data_analisis["distrito_categoria"].value_counts()

distrito_categoria
Lima Top        26808
Lima Moderna     4563
Lima Sur         1907
Lima Norte       1433
Lima Centro      1120
Lima Este         972
Name: count, dtype: int64

In [99]:
# Se crean columnas categóricas para los rangos de precio y área, que se usarán en los filtros de las pestañas.
data_analisis['precio_pen'] = pd.to_numeric(data_analisis['precio_pen'], errors='coerce')
data_analisis['precio_usd'] = pd.to_numeric(data_analisis['precio_usd'], errors='coerce')
data_analisis['area'] = pd.to_numeric(data_analisis['area'], errors='coerce')

In [100]:
# Bins y labels para precio de venta (Dólares)
bins_venta = [-1, 50000, 100000, 200000, 500000, float('inf')]
labels_venta = ["Hasta $ 50k", "De $ 50k a $ 100k", "De $ 100k a $ 200k", "De $ 200k a $ 500k", "De $ 500k a más"]
data_analisis['precio_venta_agp'] = pd.cut(data_analisis['precio_usd'], bins=bins_venta, labels=labels_venta, right=False)

# Bins y labels para área (m²)
bins_area = [-1, 50, 100, 200, 300, float('inf')]
labels_area = ["Hasta 50m2", "De 50m2 a 100m2", "De 100m2 a 200m2", "De 200m2 a 300m2", "De 300m2 a más"]
data_analisis['area_agp'] = pd.cut(data_analisis['area'], bins=bins_area, labels=labels_area, right=False)

## Estacionamiento
data_analisis["estacionamiento_gp"] = data_analisis["estacionamientos"].apply(lambda x: "Si" if x > 0 else "No")

In [103]:
# Bins y labels para precio de alquiler (Soles)
bins_alquiler = [-1, 1000, 2500, 5000, 10000, float('inf')]
labels_alquiler = ["Hasta S/ 1000", "De S/ 1000 a S/ 2500", "De S/ 2500 a S/ 5000", "De S/ 5000 a S/ 10000", "De S/ 10000 a más"]
data_analisis['precio_alquiler_agp'] = pd.cut(data_analisis['precio_pen'], bins=bins_alquiler, labels=labels_alquiler, right=False)

In [101]:
data_analisis.head()

,fuente,inmueble,operacion,distrito,zona,distrito_oficial,zona,direccion,precio,soles,...,banios,estacionamientos,tiene_estacionamientos,detalle,enlace,fecha,file_origin,precio_venta_agp,area_agp,estacionamiento_gp
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,lima cercado,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima","S/ 1,800",1800.0,...,2,NaN,0,Bonito departamento de estreno ubicado en Av. ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,NaN,De 50m2 a 100m2,No
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,lima cercado,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima","S/ 2,000 · USD 575",2000.0,...,2,NaN,0,Alquiler de Departamento en Cercado• Av. Venez...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,Hasta $ 50k,De 50m2 a 100m2,No
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,lima cercado,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado","S/ 1,800",1800.0,...,2,NaN,0,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,NaN,De 50m2 a 100m2,No
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,lima cercado,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima","S/ 2,000",2000.0,...,2,NaN,0,Moderno departamento con buenos acabados en pi...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,NaN,De 50m2 a 100m2,No
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,lima cercado,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima","S/ 1,800",1800.0,...,2,1.0,1,Alquiler de departamento con cochera en Centra...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...,NaN,De 50m2 a 100m2,Si


In [93]:
data_analisis["distrito_oficial"].value_counts()

distrito_oficial
miraflores                 8038
santiago de surco          7866
san isidro                 4847
san borja                  2258
la molina                  2105
barranco                   1694
san miguel                 1281
surquillo                   863
chorrillos                  725
jesus maria                 695
magdalena                   631
pueblo libre                577
lima cercado                574
lince                       516
ate vitarte                 408
san martin de porres        365
los olivos                  345
la victoria                 282
comas                       279
punta hermosa               275
san juan de lurigancho      260
carabayllo                  218
san bartolo                 214
san juan de miraflores      184
brena                       131
lurin                       113
chosica lurigancho           96
rimac                        93
puente piedra                91
villa el salvador            90
santa maria del mar    

In [104]:
data_analisis.columns

Index(['fuente', 'inmueble', 'operacion', 'distrito', 'zona',
       'distrito_oficial', 'zona', 'direccion', 'precio', 'soles', 'dolares',
       'precio_pen', 'precio_usd', 'variacion', 'mantenimiento',
       'tiene_mantenimiento', 'caracteristica', 'area', 'area_min', 'area_max',
       'area_promedio', 'precio_m2_pen', 'precio_m2_usd', 'dormitorio',
       'domitorio_min', 'domitorio_max', 'baños', 'banios', 'estacionamientos',
       'tiene_estacionamientos', 'detalle', 'enlace', 'fecha', 'file_origin',
       'precio_venta_agp', 'area_agp', 'estacionamiento_gp',
       'precio_alquiler_agp'],
      dtype='object')

In [105]:
# Orden final
cols_final = [
    'fuente','inmueble','operacion','distrito', 'zona', 'distrito_oficial', 'zona','direccion',
    'precio','soles','dolares','precio_pen','precio_usd','variacion',
    'mantenimiento','tiene_mantenimiento','caracteristica',
    'area','area_min','area_max','area_promedio', 'area_agp', 'precio_m2_pen','precio_m2_usd', 'precio_venta_agp', 'precio_alquiler_agp',
    'dormitorio','domitorio_min','domitorio_max','banios', 'estacionamientos','tiene_estacionamientos', 'estacionamiento_gp',
    'detalle','enlace','fecha','file_origin'
]
data_analisis = data_analisis[cols_final].copy()

In [106]:
data_analisis.head()

,fuente,inmueble,operacion,distrito,zona,zona,distrito_oficial,zona,zona,direccion,...,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima",...,3.0,NaN,2,NaN,0,No,Bonito departamento de estreno ubicado en Av. ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima",...,3.0,NaN,2,NaN,0,No,Alquiler de Departamento en Cercado• Av. Venez...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado",...,3.0,NaN,2,NaN,0,No,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima",...,3.0,NaN,2,NaN,0,No,Moderno departamento con buenos acabados en pi...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima",...,3.0,NaN,2,1.0,1,Si,Alquiler de departamento con cochera en Centra...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...


In [107]:
print("\nData Final creada satisfacotiramente!")

# Export
out_path = DATA_OUT / "data_dondevivir_analisis.csv"
data_analisis.to_csv(out_path, sep="|", index=False)
print("\nData Final creada satisfactoriamente ->", out_path)


Data Final creada satisfacotiramente!

Data Final creada satisfactoriamente -> C:\Users\PC\Desktop\Proyectos\Proyectos_Py\1.Analisis Vivienda\Analisis_Vivienda\data\processed\data_dondevivir_analisis.csv


In [108]:
data_analisis.head()

,fuente,inmueble,operacion,distrito,zona,zona,distrito_oficial,zona,zona,direccion,...,domitorio_min,domitorio_max,banios,estacionamientos,tiene_estacionamientos,estacionamiento_gp,detalle,enlace,fecha,file_origin
0,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Avenida Venezuela 2575\nLima Cercado, Lima",...,3.0,NaN,2,NaN,0,No,Bonito departamento de estreno ubicado en Av. ...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
1,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Av. Venezuela 2575\nLima Cercado, Lima",...,3.0,NaN,2,NaN,0,No,Alquiler de Departamento en Cercado• Av. Venez...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
2,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Avenida Colonial 3046\nLa Colonial, Lima Cercado",...,3.0,NaN,2,NaN,0,No,Se Alquila Departamento bien iluminado con Bal...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
3,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Av. Mariano Cornejo 1848\nLima Cercado, Lima",...,3.0,NaN,2,NaN,0,No,Moderno departamento con buenos acabados en pi...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
4,adondevivir,departamentos,alquiler,Lima Cercado,Lima Centro,Lima Centro,lima cercado,Lima Centro,Lima Centro,"Av. Colonial 2140, Lima 15081\nLima Cercado, Lima",...,3.0,NaN,2,1.0,1,Si,Alquiler de departamento con cochera en Centra...,https://www.adondevivir.com/propiedades/clasif...,2025-12-03,data_alquiler_departamentos_adondevivir_Lima C...
